# 1. Fuentes de Datos

In [106]:


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# PASO 1 CARGAR LOS DATOS

#Definimos las columnas qur vamos a usar el las tablas : Falta explicacion
a_cols = ['anime_id','name']
r_cols =['user_id','anime_id','rating']

#Cargamos los Dataframes y les aplicamos formato
anime_df = pd.read_csv("../data/anime.csv",usecols=a_cols)
rating_df = pd.read_csv("../data/rating.csv",sep=',',usecols=r_cols)

#Vemos las 5 primeras entradas de las dos tablas
print(anime_df.head())
print(rating_df.head())


#Vemos la informacion de las tablas 
anime_df.info()
rating_df.info()    



   anime_id                              name
0     32281                    Kimi no Na wa.
1      5114  Fullmetal Alchemist: Brotherhood
2     28977                          Gintama°
3      9253                       Steins;Gate
4      9969                     Gintama&#039;
   user_id  anime_id  rating
0        1        20      -1
1        1        24      -1
2        1        79      -1
3        1       226      -1
4        1       241      -1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   anime_id  12294 non-null  int64 
 1   name      12294 non-null  object
dtypes: int64(1), object(1)
memory usage: 192.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memo

In [ ]:

# PASO 2 FILTRADO DE DATOS
# Quitamos los ratings invalidos, es decir, los que no tienen una evaluacion como tal 
ratings_filtrados = rating_df[rating_df['rating'] != -1]
#print(ratings_filtrados.head())


#FILTRADO DE DATOS 
#Los animes que nos interesan del ejercicio son los que poseen más de 100 calificaciones
#Por ende primero necesitamos saber cuantas calificaciones tiene cada anime
# Creamos una variable y de los ratings filtrados guardamos cuantos ratings tienen
#cada anime agrupado por su id y contando su rating
anime_counts = ratings_filtrados.groupby('anime_id')['rating'].count()
#print(anime_counts) 

#Necesitamos animes más populares con más de 100 calificaciones. 
#Usamos los datos obtenidos recientemente
animes_populares = anime_counts[anime_counts > 100].index # index es el id del dataframe 
#Ahora necesitamos usar estos animes populares para filtrar los ratings
# No nos sirve de nada tener ratings de animes que no sean estos animes populares
# con más de 100 reviews asi que los eliminamos 
rating_df = ratings_filtrados[ratings_filtrados['anime_id'].isin(animes_populares)]

# Lo mismo con los usuarios que no tengan mas de 5 ratings
# Nos interesa quedarnos con usuarios que hayan calificado al menos 5 animes
counts_user = rating_df['user_id'].value_counts() # Contamos cuantos ratings tiene cada usuario
# Filtramos el dataframe de ratings para quedarnos solo con los usuarios que tengan 5 o más ratings
rating_df = rating_df[rating_df['user_id'].isin(counts_user[counts_user >= 5].index)]       

print(rating_df.head())

     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9


In [108]:
# PASO 3 ELIMINAR DUPLICADOS 
#Identifica filas duplicadas en un DataFrame de ratings (evaluaciones) basándose en las columnas user_id y anime_id,
#marcando como True los duplicados excepto la última ocurrencia.
duplicates_ratings = rating_df.duplicated(subset=['user_id','anime_id'],keep='last')

# Eliminar duplicados directamente, conservando la última ocurrencia
cleaned_df = rating_df.drop_duplicates(subset=['user_id','anime_id'], keep='last')


# 3. Verificar resultado
print(f"Filas originales: {len(rating_df)}")
print(f"Filas después de limpiar: {len(cleaned_df)}")





Filas originales: 6194382
Filas después de limpiar: 6194375


In [109]:

# PASO 4 LECTURA E INTERPRETACION DE RESULTADOS ESTADÍSTICOS    




# Analizamos la distribución de la cantidad de calificaciones por anime
counts_per_anime = rating_df['anime_id'].value_counts()
print(counts_per_anime.describe())
# Calcular e imprimir percentiles específicos 
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_anime.quantile(p/100)))

# Analizamos la distribución de la cantidad de calificaciones por usuario
counts_per_user = rating_df['user_id'].value_counts()
print(counts_per_user.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_user.quantile(p/100)))


count     4286.000000
mean      1445.259449
std       2495.443292
min         90.000000
25%        207.000000
50%        533.500000
75%       1515.750000
max      33454.000000
Name: count, dtype: float64
P50 = 533
P75 = 1515
P90 = 3682
P95 = 5947
P99 = 12316
count    60925.000000
mean       101.672253
std        133.470987
min          5.000000
25%         22.000000
50%         56.000000
75%        129.000000
max       2848.000000
Name: count, dtype: float64
P50 = 56
P75 = 129
P90 = 245
P95 = 347
P99 = 650


In [ ]:
# PASO 5 Identificacion de Outliers
# Un outlier es un dato que se encuentra muy alejado del resto de los datos en un conjunto.
# Tenemos que eliminarlos para evitar que afecten negativamente a nuestro modelo de recomendacion.
MAX_RATINGS_USER = counts_per_user.quantile(0.80) # Definimos un umbral para usuarios con demasiadas calificaciones (outliers)
# Filtramos los usuarios que tienen entre 5 y MAX_RATINGS_USER calificaciones
valid_users = counts_user[
    (counts_user >= 5) &
    (counts_user <= MAX_RATINGS_USER)
].index

MAX_RATINGS_ANIME = counts_per_anime.quantile(0.80) # Definimos un umbral para animes con demasiadas calificaciones (outliers)
# Filtramos los animes que tienen entre 100 y MAX_RATINGS_ANIME calificaciones
valid_animes = counts_per_anime[
    (counts_per_anime >= 100) &
    (counts_per_anime <= MAX_RATINGS_ANIME)
].index


# Filtramos el DataFrame de ratings para quedarnos solo con los usuarios y animes válidos   
rating_df = rating_df[
    rating_df['user_id'].isin(valid_users) &
    rating_df['anime_id'].isin(valid_animes)
]
print(rating_df.head())




     user_id  anime_id  rating
161        3       341       6
167        3      1121       7
168        3      1122       7
176        3      1764       6
178        3      2201       7


In [111]:
# PASO 6 VOLVEMOS A LEER E INTERPRETAR RESULTADOS ESTADÍSTICOS   
# Analizamos la distribución de la cantidad de calificaciones por anime
counts_per_anime = rating_df['anime_id'].value_counts()
print(counts_per_anime.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_anime.quantile(p/100)))
# Analizamos la distribución de la cantidad de calificaciones por usuario
counts_per_user = rating_df['user_id'].value_counts()
print(counts_per_user.describe())
# Calcular e imprimir percentiles específicos
for p in [50, 75, 90, 95, 99]:
    print(f"P{p} =", int(counts_per_user.quantile(p/100)))

count    3429.000000
mean      157.584135
std       177.087037
min         4.000000
25%        37.000000
50%        84.000000
75%       212.000000
max      1116.000000
Name: count, dtype: float64
P50 = 84
P75 = 212
P90 = 408
P95 = 559
P99 = 771
count    42285.000000
mean        12.778905
std         13.602780
min          1.000000
25%          3.000000
50%          8.000000
75%         18.000000
max        112.000000
Name: count, dtype: float64
P50 = 8
P75 = 18
P90 = 32
P95 = 41
P99 = 61


In [112]:
# PASO 7 CREACION DE LA MATRIZ DE INTERACCION USUARIO-ANIME
anime_ratings = rating_df.pivot_table(index='user_id', columns='anime_id', values='rating', fill_value=0)
print(rating_df.head())

hunter_en_columnas = 11061 in anime_ratings.columns
print(f"¿Hunter X Hunter (11061) está en las columnas? {hunter_en_columnas}")







     user_id  anime_id  rating
161        3       341       6
167        3      1121       7
168        3      1122       7
176        3      1764       6
178        3      2201       7
¿Hunter X Hunter (11061) está en las columnas? False


In [ ]:
# PASO 8 CREAR LAS RECOMENDACIONES DE ANIME HUNTER X HUNTER CON ID 11061

# 1. Cálculo de la similitud entre los animes
similarity_matrix = cosine_similarity(anime_ratings.T)
anime_similarity_df = pd.DataFrame(similarity_matrix, index=anime_ratings.columns, columns=anime_ratings.columns)

# 2. Obtenemos la similitud para Hunter X Hunter
hunter_id = 11061 
similarities = anime_similarity_df[hunter_id]

# 3. Ordenamos y filtramos
recommended_animes = similarities.sort_values(ascending=False).drop(hunter_id)

# 4. Convertimos a DataFrame y unimos con los nombres
recommendations_df = recommended_animes.head(10).reset_index()
recommendations_df.columns = ['anime_id', 'similarity_score']

# 5. Unimos con los nombres de los animes
recommendations_with_names = recommendations_df.merge(anime_df, on='anime_id', how='left')

# 6. Mostramos los resultados con nombres
print("Top 10 animes recomendados para Hunter x Hunter:")
print(recommendations_with_names[['anime_id', 'name', 'similarity_score']])




#HUNTER X HUNTER NO SE ENCUNETRA EN LA MATRIZ DE LOS ANIMES PORQUE NO CUMPLE CON LOS REQUISITOS DE FILTRADO
#PARA QUE UN ANIME SE ENCUENTRE EN LA MATRIZ DEBE TENER MÁS DE 100 RATINGS Y HUNTER X HUNTER TIENE 93 RATINGS
#¿QUE HAGO? 



KeyError: 11061